In [90]:
import tensorflow as tf
import numpy as np
import random as rn

In [91]:
#M denotes the size of the message; k denotes the number of bits required to transmit the message; 
#n_channel denotes the number of channel uses required to transmit the message; R =k/n_channel is the message rate;
#EbNodB denotes the signal-to-noise ratio at the receiver in dBm
M = 25
k = np.log2(M)
k = int(k)
n_channel = 7
R = k/n_channel
EbNodB = 5
EbNo=10.0**(EbNodB/10.0)
noise_mean = 0
noise_std = np.sqrt(1/(2*R*EbNo))

In [92]:
#generate training data 
N = 10000
l = np.random.randint(M,size=N)

In [93]:
#create one hot encoded vectors of size M for training data
data = []
for i in l:
    temp = np.zeros(M)
    temp[i] = 1
    data.append(temp)
    
data = np.array(data)

In [94]:
#generate testing data 
Nt = 450
test_l = np.random.randint(M,size=Nt)

In [96]:
#create one hot encoded vectors of size M for testing data
test_data = []
for i in test_l:
    temp = np.zeros(M)
    temp[i] = 1
    test_data.append(temp)
    
test_data = np.array(test_data)

In [97]:
#corrupt the channel with Gaussian noise
def gaussian_noise_layer(input_layer, std):
    noise = tf.random_normal(shape=tf.shape(input_layer), mean=noise_mean, stddev=noise_std, dtype=tf.float32) 
    return input_layer + noise

In [98]:
#build the autoencoder
num_inputs=M
num_hid1=n_channel
num_hid2=num_hid1 
num_output=num_inputs
lr=0.01
actf=tf.nn.relu

In [99]:
S=tf.placeholder(tf.float32,shape=[None,num_inputs])

initializer=tf.variance_scaling_initializer()

w1=tf.Variable(initializer([num_inputs,num_hid1]),dtype=tf.float32)
w2=tf.Variable(initializer([num_hid1,num_hid2]),dtype=tf.float32)
w3=tf.transpose(w1)

b1=tf.Variable(tf.zeros(num_hid1))
b2=tf.Variable(tf.zeros(num_hid2))
b3=tf.Variable(tf.zeros(num_output))

hid_layer1=tf.nn.relu(tf.matmul(S,w1)+b1)

channel = gaussian_noise_layer(hid_layer1, noise_std)

hid_layer2=tf.nn.relu(tf.matmul(channel,w2)+b2)

output_layer=tf.nn.softmax(tf.matmul(hid_layer2,w3)+b3)

In [100]:
loss=tf.reduce_mean(tf.square(output_layer-S))

optimizer=tf.train.AdamOptimizer(lr)
train=optimizer.minimize(loss)

init=tf.global_variables_initializer()

In [101]:
#train and test autoencoder; find the average bit error rate
episodes = 10

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for e in range(episodes):
        for i in range(N):
            input_ = data[i]
            sess.run(train, feed_dict={S: [input_]})
            
    no_errors_avg = 0
    for j in range(Nt):
        input_ = test_data[j]
        out_ = []
        pred_final_signal=sess.run(output_layer, feed_dict = {S: [input_]})
        pred_output = np.argmax(pred_final_signal,axis=1)
        temp = np.zeros(M)
        temp[np.int(pred_output)] = 1
        out_.append(temp)
        out_ = np.array(out_)
        no_errors = (out_ != input_)
        no_errors =  no_errors.astype(int).sum()
        no_errors_avg += no_errors
    ber = no_errors_avg / Nt
    print ('For SNR',EbNodB,'dBm, the BER is:',ber)


For SNR 5 dBm, the BER is: 0.0
